# Introduction

Writing effective algorithms with Python requires comptence with the <span style="font-family:'Courier New'">numpy</span> package, which enables:
- Fast execution
- Minimum memory footprint

Understanding <span style="font-family:'Courier New'">numpy</span>, its speed, and how to use it more effectively requires a deeper examination of how variables are handled in memory.    

This Jupyter notebook covers the essential basis of <span style="font-family:'Courier New'">numpy</span> syntax and methods to get you on your way to writing faster code that uses less memory.  It focuses on basic techniques that are frequently useful in writing algorithms.

# <span style="font-family:'Courier New'">numpy</span> versus Python Lists

The first thing to understand is that <code>numpy</code> derives its speed from storing elements of arrays in contiguous blocks of memory and relying on the fast C programming language.

![contiguous_memory](images/numpy_vs_list.jpg)

While the advantages of faster execution with <code>numpy</code> are enormous a downside is that changing an array often causes it to be re-instantiated because, in part, the current contiguous memory block is no longer appropriate and so a new block of memory needs to be found and set up.

# Speed: Establish <span style="font-family:'Courier New'">numpy</span> Variables Once <a class="anchor" id="instan_numpy-once">

Any of these actions cause a <code>numpy</code> array to be re-instantiated, which takes a non-neglible amount of time:
    
- Concatenating <span style="font-family:'Courier New'">numpy</span> arrays with <code>np.concatenate()</code>
- Appending values to <span style="font-family:'Courier New'">numpy</span> arrays with <code>np.append()</code>
- Using <code>numpy np.vstack()</code> or <code>np.hstack()</code>
- Change the data type of a <span style="font-family:'Courier New'">numpy</span> array with <code>ndarray.astype()</code>
- Resize a <span style="font-family:'Courier New'">numpy</span> array with <code>np.resize()</code>

Performing any one of these operations once on an array is fine.  Do not, however, perform any of these commands within a loop so that these operations are repeated many times.  Applying these commands multiple times within a loop is a poor idea: find a better, faster way.

Alternate approaches to constructing a <code>numpy</code> array for which all the data is not immediately available include these:

- Determine the required _size_, _shape_, and _data type_ of an array and establish it once by using either <code>np.zeros()<code> or <code>np.empty()<code>.  (The former is usually the better choice.)  Then, fill the reserved space with values are they are created.

- Accumulate data with a Python list (or list of lists) and then, when all data is accumulated, convert the data once to a <code>numpy</code> array.

Assume in the example below that we are filling a <span style="font-family:'Courier New'">numpy</span> array with computed values, which I will simulate with random numbers.  The cells below illustrate methods that are slow because of repeated use of <code>numpy</code> array reinstantiation and faster methods along the lines of the bullets immediately above.  

In [1]:
import numpy as np
import time

In [2]:
nrows = 10000
ncols = 10

In [3]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.vstack((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.22185564041137695
(10000, 10)


In [4]:
start = time.time()
np_arr = np.random.rand(ncols,1)
for i in range(nrows-1):
    np_arr = np.hstack((np_arr, np.random.rand(ncols,1)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.23386001586914062
(10, 10000)


In [9]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.append(np_arr, np.random.rand(1,ncols), axis=0)
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.2251126766204834
(10000, 10)


In [8]:
start = time.time()
np_arr = np.random.rand(1,ncols)
for i in range(nrows-1):
    np_arr = np.concatenate((np_arr, np.random.rand(1,ncols)))
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.2072908878326416
(10000, 10)


It is much faster to create a numpy array of sufficient size once to reserve space and just replace its values as the algorithm progresses.

In [11]:
np_res = np.zeros((nrows, ncols))

In [12]:
start = time.time()
for i in range(nrows):
    new_row = np.random.rand(1,ncols)
    np_res[i] = new_row
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_res.shape)

Execution time with reserved ndarray: 0.01495671272277832
(10000, 10)


If you do feel the need to append to a data structure as an algorithm progresses without defining a <code>numpy</code> array to reserve space, then accumulating data initially in a Python list before converting to a <code>numpy</code> is much faster.

In [14]:
start = time.time()
np_arr = []
for i in range(nrows):
    np_arr.append(np.random.rand(1,ncols)) 
np_arr = np.array(np_arr)
print(f'Execution time with reserved ndarray: {time.time() - start}')
print(np_arr.shape)

Execution time with reserved ndarray: 0.018949031829833984
(10000, 1, 10)


## Selecting Elements from <code>numpy</code> Arrays 

Before we continue with the topic of writing faster code, let's refresh or learn about some very useful <code>numpy</code> methods.

- <span style="font-family:'Courier New'">np.min()</span>
- <span style="font-family:'Courier New'">np.max()</span>
- <span style="font-family:'Courier New'">np.argmin()</span>
- <span style="font-family:'Courier New'">np.argmax()</span>

Algorithms frequently require that either the minimum or maximum elements be selected from an array/list or, in a more complex manner, the best element fitting particular criteria is sought.

One one just find the least or greatest array elements using the <code>np.min()</code> or <code>np.max()</code> methods, respectively.

In [2]:
x = np.random.randint(0,10,(10,))
x

array([4, 1, 0, 5, 3, 1, 8, 5, 2, 2])

In [16]:
print(x)
print(x.min(), np.min(x))
print(x.max(), np.max(x))

[7 8 0 8 4 2 0 2 3 4]
0 0
8 8


One might also find the leat and greatest elements using the <code>np.argmin()</code> or <code>np.argmax()</code> methods, respectively, although this requires a second statement to actually retrieve the element values.

In [17]:
idx_min = x.argmin()
idx_max = x.argmax()
print(x)
print(idx_min, x[idx_min])
print(idx_max, x[idx_max])

[7 8 0 8 4 2 0 2 3 4]
2 0
1 8


Despite needing a second statement to obtain a value, knowing the index of a minimum/maximum is quite useful when one must select multiple elements from an array and keep track of which elements have been selected so that they are not selected again.  This is the focus of a subsequent section in this Jupyter notebook.

The <code>np.argsort()</code> method can be useful to find the element from a list that, rather than being the least or greatest element, is the largest (smallest) item smaller (larger) than some upper (lower)limit.

In [1]:
import numpy as np

In [3]:
idx_sort = np.argsort(x)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

x: [4 1 0 5 3 1 8 5 2 2]
idx_sort: [2 1 5 8 9 4 0 3 7 6]
x[idx_sort]: [0 1 1 2 2 3 4 5 5 8]


In [4]:
idx_sort_reverse = np.flip(idx_sort)
idx_sort_reverse

array([6, 7, 3, 0, 4, 9, 8, 5, 1, 2], dtype=int64)

In [5]:
x[idx_sort_reverse]

array([8, 5, 5, 4, 3, 2, 2, 1, 1, 0])

In [6]:
idx_sort_reverse2 = idx_sort[::-1]
idx_sort_reverse2, x[idx_sort_reverse2]

(array([6, 7, 3, 0, 4, 9, 8, 5, 1, 2], dtype=int64),
 array([8, 5, 5, 4, 3, 2, 2, 1, 1, 0]))

In [7]:
# Find the largest element less than 5
i = -1
while x[idx_sort[i+1]] < 5 and i+1 < x.shape[0]:
    i += 1
print(i, idx_sort[i], x[idx_sort[i]])

6 0 4


Recall one method for sorting in descending order.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = np.flip(idx_sort)
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

This is another method, although it is perhaps less intuitive.

In [ ]:
idx_sort = np.argsort(x)
idx_sort = idx_sort[::-1]
print(f'x: {x}')
print(f'idx_sort: {idx_sort}')
print(f'x[idx_sort]: {x[idx_sort]}')

## Application: The Cell Tower Problem

In [8]:
import random

def setup():
    prob_size = 100000
    data = [random.random() for _ in range(prob_size)]
    budget = 5.0
    return data, budget

### Python List with Deletion of Used Elements

### With a <code>for</code> Loop

In [9]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

for t in towers:
    if sum(towers_to_pick) + t <= budget:
        towers_to_pick.append(towers[0])
    del towers[0]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 5.159128242633061 
Execution time: 0.45479726791381836 seconds 
Towers selected: [0.521292076918491, 0.9039285726772568, 0.2234210440759059, 0.7620340967233455, 0.6446713242303519, 0.8444811333070722, 0.8440404570800887, 0.415259537620549]


In [10]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

for t in towers:
    if sum(towers_to_pick) + t <= budget:
        towers_to_pick.append(towers.pop(0))
    else:
        _ = towers.pop(0)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 5.824380967695865 
Execution time: 0.40062952041625977 seconds 
Towers selected: [0.8063494533871537, 0.48522924765684494, 0.7766759064743349, 0.1021062758655169, 0.40822842125707615, 0.4766958542734133, 0.6120018902578022, 0.08291375080677332, 0.8401722693022138, 0.39857558632612, 0.8354323120886157]


In [11]:
budget

5.0

### With a <code>while</code> Loop

Using a <code>while</code> loop in this situation in a way to iterate through all the data is an inferior method to using a <code>for</code> loop, but the examples are shown for speed comparison in any case.

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []

while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers.pop(0))
    else:
        _ = towers.pop(0)

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

In [ ]:
import random
import time

towers, budget = setup()
time_start = time.time()

towers_to_pick = []
while sum(towers_to_pick) < budget and len(towers) > 0:
    if sum(towers_to_pick) + towers[0] <= budget:
        towers_to_pick.append(towers[0])
    del towers[0]
    # towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

### <code>numpy</code> with Slices to Delete Used Elements

In [12]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

towers_to_pick = np.array([])

while towers_to_pick.sum() < budget and towers.shape[0] > 0:
    if towers_to_pick.sum() + towers[0] <= budget:
        towers_to_pick = np.append(towers_to_pick, towers[0])
    towers = towers[1:]

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick}')

Investment: 4.99999251169534 
Execution time: 0.41970157623291016 seconds 
Towers selected: [3.15036118e-02 7.92562909e-01 8.44337737e-01 7.67850078e-01
 8.12229136e-01 2.61588378e-01 6.37617191e-01 7.58163581e-02
 4.15168268e-01 3.01501804e-01 3.82622392e-03 3.49692121e-02
 5.36431523e-04 9.27625376e-03 3.53389752e-03 5.36277993e-03
 1.73563622e-03 1.06410941e-04 7.41229443e-05 3.96071412e-04]


### Efficient <code>numpy</code> with Reserved Memory for Array

In [13]:
import numpy as np

towers, budget = setup()
towers = np.array(towers)
time_start = time.time()

''' Reserve space for solution of maximum possible size '''
towers_to_pick = np.zeros(towers.shape[0], dtype=np.float32)  # do not use np.empty()!!!

j = 0  # counter for number of elements packed and the index of the next element to be packed
for cost in towers:
    if cost <= budget:
        towers_to_pick[j] = cost
        budget -= cost
        j += 1

print(f'Investment: {sum(towers_to_pick)} \nExecution time: {time.time() - time_start} seconds \nTowers selected: {towers_to_pick[:j]}')

Investment: 4.999996112390363 
Execution time: 0.026012659072875977 seconds 
Towers selected: [1.9919200e-01 9.1937554e-01 1.8774213e-01 1.5927279e-01 5.1257706e-01
 2.2607550e-01 4.3258750e-01 6.8232965e-01 1.8455960e-01 5.8031601e-01
 4.2420509e-01 1.8288933e-01 2.1424871e-02 2.7501827e-01 2.5330577e-03
 1.1646652e-03 8.5393032e-03 2.2012500e-05 2.5368619e-05 1.3688307e-04
 9.4722891e-06]


## Boolean Masks

A Boolean (<code>True</code>/<code>False</code>) array can be used to filter out values from a <code>numpy</code> array.  Array elements whose position coincide with a <code>False</code> are filtered out.

### Example 1

In [14]:
size = 5
x = np.arange(size)
x

array([0, 1, 2, 3, 4])

In [15]:
mask_x = np.array([True if i%2==1 else False for i in range(size)])
mask_x

array([False,  True, False,  True, False])

In [16]:
print(x[mask_x])

[1 3]


### Example 2

In [17]:
y = np.arange(size**2).reshape(size,size)
y

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [18]:
mask_y = np.array([True if i%2==1 else False for i in range(size**2)]).reshape(5,5)
mask_y

array([[False,  True, False,  True, False],
       [ True, False,  True, False,  True],
       [False,  True, False,  True, False],
       [ True, False,  True, False,  True],
       [False,  True, False,  True, False]])

In [19]:
print(y[mask_y])

[ 1  3  5  7  9 11 13 15 17 19 21 23]


### Example 3: Select Array Rows

In [21]:
row_mask = [False, True, False, True, True]

In [22]:
y[row_mask]

array([[ 5,  6,  7,  8,  9],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [23]:
y[row_mask,:]

array([[ 5,  6,  7,  8,  9],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

### Example 4: Select Array Columns

In [24]:
col_mask = [True, False, False, False, True]

In [25]:
y[:,col_mask]

array([[ 0,  4],
       [ 5,  9],
       [10, 14],
       [15, 19],
       [20, 24]])

### Example 5: Select Array Rows and columns

In [26]:
y[row_mask,:][:,col_mask]

array([[ 5,  9],
       [15, 19],
       [20, 24]])

In [27]:
y[row_mask,col_mask]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (3,) (2,) 

### Example 6: Select on Criterion

In [28]:
z = np.random.random(size = (size,))
z

array([0.45389529, 0.14136942, 0.45470641, 0.60656471, 0.17254491])

In [29]:
z >= 0.5

array([False, False, False,  True, False])

In [30]:
mask_z = (z >= 0.5)
mask_z

array([False, False, False,  True, False])

In [31]:
z[mask_z]

array([0.60656471])

In [32]:
z[z >= 0.5]

array([0.60656471])

## Application 2: Traveling Salesperson Problem

In this problem, the task is to maintain the original data in its original instantiation without deleting the data pertaining to the destinations already included in the Traveling Salesperson's route.

### A Traveling Salesperson Problem (TSP) Greedy Algorithm

Randomly select a location to start.  Assume that we select Location 1.

- Loop until all locations visited
  - For each location, choose the next location to be closest possible next location of locations not yet visited
  
![AlgoStep1](images/m1.jpg)
![AlgoStep2](images/m2.jpg)
![AlgoStep3](images/m3.jpg)
![AlgoStep4](images/m4.jpg)
![AlgoStep5](images/m5.jpg)

Route: 1-2-0-4-3-1

This algorithm could be implemented by deleting an array column when each next stop location is determined.  One could, alternately, "mask" out those columns so that locations already in the route could not be revisited.  The latter approach avoids needing to re-instantiate the array multiple times.

#### Set up the data

In [33]:
# create distance matrix
nloc = 10
dist = np.random.rand(nloc,nloc)
dist = np.triu(dist,k=0)
for i in range(1,nloc):
    for j in range(0, i):
        dist[i,j] = dist[j,i]
for i in range(nloc):
    dist[i,i]=0.0
dist

array([[0.        , 0.17131913, 0.763575  , 0.83418007, 0.53254318,
        0.2609018 , 0.24543672, 0.54181851, 0.26402062, 0.58427671],
       [0.17131913, 0.        , 0.48608228, 0.66649735, 0.16160381,
        0.78131542, 0.26657276, 0.92267292, 0.10332775, 0.1114849 ],
       [0.763575  , 0.48608228, 0.        , 0.54738361, 0.39671481,
        0.84975135, 0.99434816, 0.88782606, 0.80703642, 0.1046208 ],
       [0.83418007, 0.66649735, 0.54738361, 0.        , 0.53790223,
        0.82184891, 0.62323072, 0.09515994, 0.44585599, 0.66215463],
       [0.53254318, 0.16160381, 0.39671481, 0.53790223, 0.        ,
        0.26229732, 0.81991608, 0.59805163, 0.53262593, 0.49493769],
       [0.2609018 , 0.78131542, 0.84975135, 0.82184891, 0.26229732,
        0.        , 0.91780919, 0.20244532, 0.1560945 , 0.5368338 ],
       [0.24543672, 0.26657276, 0.99434816, 0.62323072, 0.81991608,
        0.91780919, 0.        , 0.74523658, 0.51248952, 0.716795  ],
       [0.54181851, 0.92267292, 0.8878260

In [34]:
''' Set up parameters '''
nloc = dist.shape[0]                      # number of locations
assert dist.shape[0] == dist.shape[1]     # ensure square distance matrix

''' Initialize random starting point '''
start = np.random.randint(0, nloc-1)      # select random starting location
sol = [start]                             # solution route in a list
cur_loc = start                           # use cur_loc to indicate current location index

''' Establish Boolean mask for the columns: True = column location not visited '''
col_mask = np.ones(nloc).astype(np.bool_) # creates array of True
col_mask[start] = False                   # cannot choose starting location as
                                          # next location

''' Create ndarray of column indices '''
col_indices = np.arange(nloc)             # create array of indices

''' Initial distance of solution '''
sol_dist = 0.0                            # initialize distance of solution

''' Execute algorithm '''
while col_mask.sum() > 0:              # continue if any True values in col_mask
    ''' Get index of next location '''
    next_loc_ind = np.argmin(dist[cur_loc][col_mask])  # get index of row minimum for
                                                       #  remaining locations
    next_loc_ind = col_indices[col_mask][next_loc_ind] # find index of minimum relative to original
                                                       #   indices (true index of location)
    
    ''' Update solution and mask '''
    sol.append(next_loc_ind)                   # append next location to solution
    col_mask[next_loc_ind] = False             # update mask for current location
    sol_dist += dist[cur_loc, next_loc_ind]    # update solution distance
    cur_loc = next_loc_ind                     # update current location

sol.append(start)       # append starting location for round trip
sol, sol_dist

([6, 0, 1, 8, 5, 7, 3, 4, 2, 9, 6], 2.013021201384613)